In [3]:
import csv
import datetime
import os
import yaml
from itertools import islice

from nupic.frameworks.opf.model_factory import ModelFactory

_NUM_RECORDS = 3000
_INPUT_FILE_PATH = "C:\Users\mossa\Nupic_doc\gymdata.csv"
_PARAMS_PATH = "C:\Users\mossa\Nupic_doc\model.yaml"

#Nupicのmodelfactoryとモデルの中身を記述したyamlファイルを使ってOPFモデルを作成する

def createModel():
    with open(_PARAMS_PATH, "r") as f:
        modelParams = yaml.safe_load(f)
    return ModelFactory.create(modelParams)



def runHotgym(numRecords):
    model = createModel()
    #予測のデータを返却する
    model.enableInference({"predictedField": "consumption"})
    with open(_INPUT_FILE_PATH) as fin:
        #ヘッダーの位置をスキップする
        reader = csv.reader(fin)
        headers = reader.next()
        reader.next()
        reader.next()
        results = []
        #itertoolsでループ処理
        for record in islice(reader, numRecords):
            #pythonの辞書形式でデータを読み込む
            #modelInputはgymdata.csvの形式から{'timestamp': '7/2/10 0:00', 'consumption': '21.2'}のような辞書になる
            modelInput = dict(zip(headers, record))
            modelInput["consumption"] = float(modelInput["consumption"])
            modelInput["timestamp"] = datetime.datetime.strptime(modelInput["timestamp"], "%m/%d/%y %H:%M")
            #HTMの実行
            result = model.run(modelInput)
            #実行結果の取り出しを行う
            #multiStepBestPredictionsは最後のデータ行で返された最適な予測に関する情報(予測値)
            #multiStepPredictionsは各予測の信頼値を含む、データの最終行のすべての予測に関する情報(信頼値)
            bestPredictions = result.inferences["multiStepBestPredictions"]
            allPredictions = result.inferences["multiStepPredictions"]
            #1ステップ後の予測
            oneStep = bestPredictions[1]
            oneStepConfidence = allPredictions[1][oneStep]
            #5ステップ後の予測
            fiveStep = bestPredictions[5]
            fiveStepConfidence = allPredictions[5][fiveStep]
            #予測値(信頼値)の形でprint
            result = (oneStep, oneStepConfidence * 100,fiveStep, fiveStepConfidence * 100)
            print "1-step: {:16} ({:4.4}%)\t 5-step: {:16} ({:4.4}%)".format(*result)
            results.append(result)
    return results


if __name__ == "__main__":
    runHotgym(_NUM_RECORDS)

1-step:             21.2 (100.0%)	 5-step:             21.2 (100.0%)
1-step:             16.4 (99.8%)	 5-step:             16.4 (99.8%)
1-step:              4.7 (99.6%)	 5-step:              4.7 (99.6%)
1-step:              4.7 (99.6%)	 5-step:              4.7 (99.6%)
1-step:              4.6 (99.4%)	 5-step:              4.6 (99.4%)
1-step:             23.5 (99.4%)	 5-step:             23.5 (99.4%)
1-step:             47.5 (99.21%)	 5-step:             47.5 (99.21%)
1-step:             45.4 (99.06%)	 5-step:             45.4 (99.06%)
1-step:             46.1 (98.87%)	 5-step:             46.1 (98.87%)
1-step:             41.5 (98.87%)	 5-step:             41.5 (98.87%)
1-step:             43.4 (98.68%)	 5-step:             43.4 (98.68%)
1-step:            45.61 (96.41%)	 5-step:             43.8 (94.31%)
1-step:             37.8 (83.42%)	 5-step:             37.8 (90.28%)
1-step:             36.6 (86.53%)	 5-step:             36.6 (90.13%)
1-step:             4.67 (79.87%)	 5-step:  

1-step:    4.65850461565 (100.0%)	 5-step:           42.388 (100.0%)
1-step:    4.65850461565 (100.0%)	 5-step:    38.9483378705 (100.0%)
1-step:    4.65850461565 (100.0%)	 5-step:         38.05986 (100.0%)
1-step:       22.2672168 (100.0%)	 5-step:          40.6145 (100.0%)
1-step:           42.388 (100.0%)	 5-step:          44.0271 (100.0%)
1-step:    38.9483378705 (99.81%)	 5-step:           41.443 (100.0%)
1-step:       43.3811162 (99.93%)	 5-step:          40.6145 (100.0%)
1-step:          40.6145 (100.0%)	 5-step:        39.882363 (100.0%)
1-step:      43.23678134 (100.0%)	 5-step:    38.9483378705 (100.0%)
1-step:           41.443 (100.0%)	 5-step:    38.9483378705 (100.0%)
1-step:          40.6145 (100.0%)	 5-step:      36.98072232 (100.0%)
1-step:       39.7376541 (100.0%)	 5-step:          40.6145 (100.0%)
1-step:    38.9483378705 (100.0%)	 5-step:      39.66635787 (100.0%)
1-step:          44.0271 (99.99%)	 5-step:      43.23678134 (100.0%)
1-step:     37.046505624 (100.0%)	

KeyboardInterrupt: 